In [1]:
%matplotlib inline
%load_ext memory_profiler 

In [2]:
from KnackpackData import n_items,weights,values,max_weight,max_value,encodeNumber,items_size_threshold,getDiff
from Timer import *
import matplotlib.pyplot as plt
import numpy as np
import random
from numba import njit,prange
t = Timer()

## Branch & Bound

Branch and Bound to technika używana w rozwiązywaniu problemów opymalizacji. Jest ona nieco "lepsza" od metod wyczerpujących (exhaustive), ponieważ dokonuje oceny już częściowo pryzgotowanych rozwiazań, a jeżeli potencjalne pozostałe rozwiazania ne generują lepszych wyników, wtedy są pomijane.
W najgorszym przypadku B&B ma wykładniczą złożoność i wygeneruje wszystkie możliwe kombinacje liści. Wtedy drzewo będzie kompletne i będzie zawierało $2^{n-1}-1$ odgałęzień. Mimo to przeważnie i tak będzie lepszym wyborem niż sprawdzanie wszystkich możliwych rozwiązań metodą Brute Force.

B&B bazuje na drzewie przestrzni stanów, gdzie każdy kolejny poziom reprezentuje ścieżki dążące od lisci do korzenia i zaweirające rozwiazania. Przy czym korzeń (poziom 0) to punkt startowy bez żadnego rozwiązania. 

W przypadku problemu plecakowego 0-1 przy $N$ dostepnych przedmiotach $k$-ty poziom reprezentuje stan z którego wybierane jest rozwiązanie (wybrane zostały przedmioty lub nie). To daje $2^{k}$ gałęzi na $k$-tym poziomie drzewa, a liście znajdują się na poziomie $N$.


W drzewie przestrzeni stanów lewe odgałęzienie powinno oznaczać użycie kolejnego przedmiotu, a prawa gałąź jego wykluczenie.

Obliczana górna granica ($Upper Bound$) obliczana jest na zasadzie dodawania wartości obecnego podzbioru przedmiotów oraz pozostałej pojemności plecaka wymnożonej przez najlepszy kolejny przedmiot:

$Upper Bound = v + (Capacity - w)*(\frac{vi+1}{wi+1})$

Przed uruchomieniem algorytmu wszystkie przedmioty powinny być posortowane w stosunku wartość ($v$) - waga ($w$): 
$\frac{v}{w}$.

### Opis
Możliwe są 2 implementacje algorytmu B&B: 
1. wybieranie gałęzi z najmniejszą wartością graniczną. Ten sposób sprawdza mniej możliwości, a co za tym idzie zajmuje mniej czasu, ale wymaga więcej pamięci.
2. wybieranie gałezi z najmniejszą wartością graniczną z nowo utworzonych gałęzi. Ten sposób wymaga mniej pamięci ale więcej możliwych rozgałęzień co wydłuża jego czas pracy.

W poniższym algorytmie została wykorzystana metoda 1.

Algorytm na początku tworzy korzeń i z niego kolejno generuje lewe i prawe odgałęzienia sprawdzając ich granice i osiągnięte wartości.

In [3]:
# Data must be sorted in value/weights ratio way.
VW_ratios = values / weights
indexes = np.flip(np.argsort(VW_ratios))

In [4]:
VW_ratios = np.take_along_axis(VW_ratios, indexes, axis=0) # Value/weight ratios
VW_weights = np.take_along_axis(weights, indexes, axis=0) # Sorted weights
VW_values = np.take_along_axis(values, indexes, axis=0) # Sorted values

In [5]:
'''
Based on:
A. Shaheen and A. Sleit, ‘‘Comparing between different approaches to solve the 0/1 Knapsack problem’’ 
Int. J. Comput. Sci. Netw. Secur., vol. 16,no. 7, p. 1. 2016.
'''

# *** Branching with smallest node. ***

class State:
    def __init__(self, level, value, weight, bound, items):
        self.level = level
        self.value = value
        self.weight = weight
        self.bound = bound
        self.items = items
               
def upper_bound(branch, max_weight, item_count, sorted_values, sorted_weights):
    
    # *This condition is not explicitely included in default B&B*
    # If child weight exceeds maximum weight its bound should be 0 or None.
    if branch.weight > max_weight:
        return 0
    else:
        bound = branch.value 
        weight = branch.weight
        level = branch.level # Level indicates amount of items in knapsack.
        
        # Calculate upper bound based on branch parameters and left capacity
        while level < item_count and weight + sorted_weights[level] <= max_weight:
            bound += sorted_values[level]
            weight += sorted_weights[level]
            level += 1
        
        # If there is any item left (best according to value/weight ratio) add it to bound.
        if level < item_count:
            bound += (max_weight - weight) * float(sorted_values[level])/ sorted_weights[level]
            
        return bound

def run():
    root = State(0, 0.0, 0.0, 0.0, []) # Initialize root as empty branch.
    root.bound = upper_bound(root, 3, 16, VW_values, VW_weights) # Calculate root upper bound.

    queue = [] # Queue to store new branches.
    queue.append(root) # First item in queue is root.

    max_value = root.value # First max_value is root value (0).

    best_items = [0]*n_items # First best items is empty.

    # Iterate till queue is not empty.
    while queue:
        # Get the smallest node value (a.k.a last value).
        current_state = queue.pop()

        # If node bound is greater than max found value then go to the next level.
        if current_state.bound > max_value:
            level = current_state.level + 1 

        # Generate left branch.
        left = State(level, current_state.value + VW_values[current_state.level], current_state.weight + VW_weights[current_state.level], 0.0, current_state.items[:])
        # Calculate new left branch upper bound.
        left.bound = upper_bound(left, max_weight, n_items, VW_values, VW_weights)
        left.items.append(level)

        # *This condition is not explicitely included in default B&B*
        # If weight doesn't exceed maximum capacity proceed to checking vlaues.
        if left.weight <= max_weight:
            # If value is greater than max found vlaue set it as new max.
            if left.value > max_value:
                max_value = left.value
                best_items = left.items
            # Append child if its upper bound is greater than max found value.
            if left.bound > max_value:
                queue.append(left)

        # Generate right branch.
        right = State(level,current_state.value, current_state.weight, 0.0, current_state.items[:])
        # Calculate new right branch upper bound.
        right.bound = upper_bound(right, max_weight, n_items, VW_values, VW_weights)

        # Right branch is considered if left branch is not present, but in general it should not be better than left (state space tree principle).
        if right.weight <= max_weight:
            if right.value > max_value:
                max_value = right.value
                best_items = right.items
            if right.bound > max_value:
                queue.append(right)

    taken_items = [0]*n_items
    
    return best_items, taken_items

In [6]:
t.start()
best_items, taken_items = run()
t.stop()

for item_idx in best_items:
    taken_items[item_idx-1] = 1
best_value = np.sum(VW_values[(np.where(np.array(taken_items) == 1))])
best_weight = np.sum(VW_weights[(np.where(np.array(taken_items) == 1))])
print("Best value: ",best_value)
print("Best weight: ",best_weight)
print("Difference (value, weight):", getDiff(n_items,best_value, best_weight))

Elapsed time: 0.0092 seconds.
Best value:  4.996682595542677
Best weight:  2.8953181580761527
Difference (value, weight): [4.457322511086659e-09, 1.9238473036864434e-09]


### Wnioski

Algorytm B&B jest dużo wydajniejszym algorytmem przeszukiwania niż metoda Brute Force, a dającym bardzo dobre wyniki. Uzyskany wynik jest równy najlepszemu znanemu w danym problemie. 
Algorytm ten bardzo dobrze sprawdza się dla małych rozmiarów problemów z niską korelacją waga-wartosć co zostanie przedstawione w dalszej części.